# Excercise 1 - user privileges

In [1]:
import sys
import mysql.connector

def myconnect(user, pw):
        conn = mysql.connector.connect( host='localhost', database='classicmodels',user=user, password=pw)
        conn.autocommit = True
        return conn   

rootconn = myconnect('root','00020297') # use root password


def sqlQuery(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        #res = cursor.fetchwarnings()
        #return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()


# Inventory - which is used to maintain the two tables products and productlines`.

In [2]:
sqlDo("DROP USER IF EXISTS Inventory") # if user exists then drop
sqlDo("CREATE USER 'Inventory' IDENTIFIED BY '123'") # cerate new user name inventory
sqlQuery("select User, Host from mysql.user") # show all user
sqlQuery("SHOW GRANTS FOR Inventory") # show privilege of user inventory
#secretconn = myconnect('Inventory','123')

[('GRANT USAGE ON *.* TO `Inventory`@`%`',)]

In [3]:
#sqlDo("GRANT UPDATE, INSERT, DELETE, SELECT  ON classicmodels.products TO Inventory")
sqlDo("GRANT ALL  ON classicmodels.products  TO Inventory")
sqlDo("FLUSH Privileges")


In [4]:
productsconn = myconnect('Inventory','123')

In [5]:
sqlQuery("Select * from  classicmodels.products limit 1 ", productsconn)

[('S10_1678',
  '1969 Harley Davidson Ultimate Chopper',
  'Motorcycles',
  '1:10',
  'Min Lin Diecast',
  'This replica features working kickstand, front suspension, gear-shift lever, footbrake lever, drive chain, wheels and steering. All parts are particularly delicate due to their precise scale and require special care and attention.',
  7933,
  Decimal('48.81'),
  Decimal('95.70'))]

In [6]:
sqlDo("GRANT ALL  ON classicmodels.productlines  TO Inventory")

In [7]:
sqlDo("FLUSH Privileges")

In [8]:
sqlQuery("Select * from  classicmodels.productlines limit 1 ", productsconn)

[('Classic Cars',
  'Attention car enthusiasts: Make your wildest car ownership dreams come true. Whether you are looking for classic muscle cars, dream sports cars or movie-inspired miniatures, you will find great choices in this category. These replicas feature superb attention to detail and craftsmanship and offer features such as working steering system, opening forward compartment, opening rear trunk with removable spare wheel, 4-wheel independent spring suspension, and so on. The models range in size from 1:10 to 1:24 scale and include numerous limited edition and several out-of-production vehicles. All models include a certificate of authenticity from their manufacturers and come fully assembled and ready for display in the home or office.',
  None,
  None)]

# Bookkeeping which make sure that all orders are payed.

In [9]:
sqlDo("DROP USER IF EXISTS bookkeeping") # if user exists then drop
sqlDo("CREATE USER 'bookkeeping' IDENTIFIED BY '1234'") # cerate new user name bookeeping
sqlQuery("select User, Host from mysql.user") # show all user
sqlQuery("SHOW GRANTS FOR bookkeeping") # show privilege of user bookkeeping

[('GRANT USAGE ON *.* TO `bookkeeping`@`%`',)]

In [10]:
sqlDo("GRANT SELECT  ON classicmodels.orders  TO Inventory")
sqlDo("FLUSH Privileges")

In [11]:
bookkeepingconn = myconnect('Inventory','123')

In [12]:
sqlQuery("Select * from  classicmodels.orders limit 1 ", bookkeepingconn)

[(10100,
  datetime.date(2003, 1, 6),
  datetime.date(2003, 1, 13),
  datetime.date(2003, 1, 10),
  'Shipped',
  None,
  363)]

## Human resources which takes care of employees and their offices

In [13]:
sqlDo("DROP USER IF EXISTS HRM") # if user exists then drop
sqlDo("CREATE USER 'HRM' IDENTIFIED BY '12345'") # cerate new user name bookeeping
sqlQuery("select User, Host from mysql.user") # show all user
sqlQuery("SHOW GRANTS FOR HRM") # show privilege of user bookkeeping

[('GRANT USAGE ON *.* TO `HRM`@`%`',)]

In [14]:
sqlDo("GRANT UPDATE, INSERT, DELETE, SELECT  ON classicmodels.employees TO HRM")
sqlDo("FLUSH Privileges")

In [15]:
hrmconn = myconnect('HRM','12345')

In [16]:
sqlQuery("SET FOREIGN_KEY_CHECKS=0; Delete from classicmodels.employees where employees.employeeNumber ='1076'", hrmconn)

Use multi=True when executing multiple statements


## Exercise 2 - logging

In [34]:
sqlDo("SET global general_log = 1; SET global log_output = 'table'") # create log file

In [35]:

sqlQuery("Explain mysql.general_log")

No result set to fetch from.


In [ ]:
### insert some data to see the recors of lof file

In [ ]:
sqlQuery("""
SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, user_host, command_type, argument 
FROM mysql.general_log
ORDER BY time DESC
LIMIT 10
""")
# investigate record of log file

## Sales - who creates the orders for the customers

In [17]:
sqlDo("DROP USER IF EXISTS sales") # if user exists then drop
sqlDo("CREATE USER 'sales' IDENTIFIED BY '0020'") # cerate new user name bookeeping
sqlQuery("select User, Host from mysql.user") # show all user
sqlQuery("SHOW GRANTS FOR employees") # show privilege of user bookkeeping

[('GRANT USAGE ON *.* TO `employees`@`%`',)]

In [18]:
sqlDo("GRANT INSERT ON classicmodels.orders TO sales")
sqlDo("FLUSH Privileges")

In [19]:
hrmconn = myconnect('sales','0020')

In [31]:
sqlDo("""
INSERT INTO classicmodels.orders
(orderDate,requiredDate,shippedDate,status,comments,customerNumber)
VALUES('2019-02-24','2019-02-26','2019-02-28','good','yes',100)
""",hrmconn)


1364 (HY000): Field 'orderNumber' doesn't have a default value


## Exercise 3 - backup and recovery

In [ ]:
C:\Program Files\MySQL\MySQL Server 8.0\bin>mysqldump -u root -p test>back.sql # create back up database command

In [ ]:
C:\Program Files\MySQL\MySQL Server 8.0\bin>mysqldump -u root -p test<back.sql  # restore database from backup